In [1]:
from scipy.stats import linregress
import pandas as pd


def process_sdg_data(input_excel_file, columns_to_drop):
    df = pd.read_excel(input_excel_file)
    df = df.drop(columns_to_drop, axis=1)
    df = df.set_index("GeoAreaName").transpose()
    return df
    

def fit_trendline(year_timestamps, data):
    result = linregress(year_timestamps, data)
    slope = round(result.slope, 3)
    r_squared = round(result.rvalue**2, 3)
    return slope, r_squared


def country_trendline(country_name):
    df = process_sdg_data("data/SG_GEN_PARL.xlsx", 
                          ["Goal", "Target", "Indicator", "SeriesCode", "SeriesDescription", "GeoAreaCode", "Reporting Type", "Sex", "Units"])
    timestamps = [int(i) for i in df.index.tolist()]
    country_data = df[country_name].tolist()
    slope, r_squared = fit_trendline(timestamps, country_data)
    return slope, r_squared


country_trendline("India")

(0.292, 0.869)

In [2]:
from fastapi import FastAPI


app = FastAPI()
app

In [3]:
@app.get("/country_trendline/{country}")
def calculate_country_trendline(country: str):
    slope, r_squared = country_trendline(country)
    return {"slope": slope, "r_squared": r_squared}

In [4]:
%%writefile chapter_11_functions.py
from scipy.stats import linregress
from fastapi import FastAPI
import pandas as pd


def process_sdg_data(input_excel_file, columns_to_drop):
    df = pd.read_excel(input_excel_file)
    df = df.drop(columns_to_drop, axis=1)
    df = df.set_index("GeoAreaName").transpose()
    return df
    

def fit_trendline(year_timestamps, data):
    result = linregress(year_timestamps, data)
    slope = round(result.slope, 3)
    r_squared = round(result.rvalue**2, 3)
    return slope, r_squared


def country_trendline(country_name):
    df = process_sdg_data("data/SG_GEN_PARL.xlsx", 
                          ["Goal", "Target", "Indicator", "SeriesCode", "SeriesDescription", "GeoAreaCode", "Reporting Type", "Sex", "Units"])
    timestamps = [int(i) for i in df.index.tolist()]
    country_data = df[country_name].tolist()
    slope, r_squared = fit_trendline(timestamps, country_data)
    return slope, r_squared


app = FastAPI()


@app.get("/country_trendline/{country}")
def calculate_country_trendline(country: str):
    slope, r_squared = country_trendline(country)
    return {"slope": slope, "r_squared": r_squared}

Overwriting chapter_11_functions.py


In [5]:
!uvicorn chapter_11_functions:app

INFO:     Started server process [84266]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 48] error while attempting to bind on address ('127.0.0.1', 8000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


In [6]:
%%writefile chapter_11_functions.py
from scipy.stats import linregress
from pydantic import BaseModel
from fastapi import FastAPI
from typing import List
import pandas as pd


def process_sdg_data(input_excel_file, columns_to_drop):
    df = pd.read_excel(input_excel_file)
    df = df.drop(columns_to_drop, axis=1)
    df = df.set_index("GeoAreaName").transpose()
    return df
    

def fit_trendline(year_timestamps, data):
    result = linregress(year_timestamps, data)
    slope = round(result.slope, 3)
    r_squared = round(result.rvalue**2, 3)
    return slope, r_squared


def country_trendline(country_name):
    df = process_sdg_data("data/SG_GEN_PARL.xlsx", 
                          ["Goal", "Target", "Indicator", "SeriesCode", "SeriesDescription", "GeoAreaCode", "Reporting Type", "Sex", "Units"])
    timestamps = [int(i) for i in df.index.tolist()]
    country_data = df[country_name].tolist()
    slope, r_squared = fit_trendline(timestamps, country_data)
    return slope, r_squared


app = FastAPI()


@app.get("/country_trendline/{country}")
def calculate_country_trendline(country: str):
    slope, r_squared = country_trendline(country)
    return {"slope": slope, "r_squared": r_squared}


class TrendlineInput(BaseModel):
    timestamps: List[int]
    data: List[float]


# @app.post("/fit_trendline/")
@app.post("/fit_trendline/", 
          summary="Fit a trendline to any data",
          description="Provide a list of integer timestamps and a list of floats")
def calculate_trendline(trendline_input: TrendlineInput):
    slope, r_squared = fit_trendline(trendline_input.timestamps,
                                     trendline_input.data)
    return {"slope": slope, "r_squared": r_squared}

Overwriting chapter_11_functions.py


In [7]:
# !uvicorn chapter_11_functions:app
import requests


response = requests.get("http://127.0.0.1:8000/country_trendline/India")
response.status_code

200

In [8]:
response.json()

{'slope': 0.292, 'r_squared': 0.869}

In [9]:
url = "http://127.0.0.1:8000/fit_trendline/"
json_data = {"timestamps": [2000, 2001, 2002],
             "data": [0.5, 0.6, 0.7]}
response = requests.post(url=url, json=json_data)
response.json()

{'slope': 0.1, 'r_squared': 1.0}